#NIFTY50 2dCNNpred

2dCNNpred implemention based on the paper trained using NIFTY50 historical data

In [1]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import output
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error

In [2]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
 
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
 
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
 
def f1macro(y_true, y_pred):
    f_pos = f1_m(y_true, y_pred)
    # negative version of the data and prediction
    f_neg = f1_m(1-y_true, 1-K.clip(y_pred,0,1))
    return (f_pos + f_neg)/2

In [3]:
def datagen(data, seq_len, batch_size, targetcol, kind):
    #As a generator to produce samples for Keras model
    batch = []
    while True:
        df = data
        input_cols = [c for c in df.columns if c != targetcol]
        index = df.index[df.index < TRAIN_TEST_CUTOFF]
        split = int(len(index) * TRAIN_VALID_RATIO)
        if kind == 'train':
            index = index[:split]   # range for the training set
        elif kind == 'valid':
            index = index[split:]   # range for the validation set
        
        # Pick one position, then clip a sequence length
        while True:
            t = random.choice(index)      # pick one time step
            n = (df.index == t).argmax()  # find its position in the dataframe
            if n-seq_len+1 < 0:
                continue # can't get enough data for one sequence length
            frame = df.iloc[n-seq_len+1:n+1]
            batch.append([frame[input_cols].values, df.loc[t, targetcol]])
            break
        # if we get enough for a batch, dispatch
        if len(batch) == batch_size:
            X, y = zip(*batch)
            X, y = np.expand_dims(np.array(X), 3), np.array(y)
            yield X, y
            batch = []

In [4]:
def testgen(data, seq_len, targetcol):
    #Return array of all test samples
    batch = []
    df=data
    input_cols = [c for c in df.columns if c != targetcol]
    # find the start of test sample
    t = df.index[df.index >= TRAIN_TEST_CUTOFF][0]
    n = (df.index == t).argmax()
    # extract sample using a sliding window
    for i in range(n+1, len(df)+1):
        frame = df.iloc[i-seq_len:i]
        batch.append([frame[input_cols].values, frame[targetcol][-1]])
    X, y = zip(*batch)
    return np.expand_dims(np.array(X),3), np.array(y)

In [5]:
url='https://raw.githubusercontent.com/mandalnilabja/soc2022/main/data/NSEI.csv'

X = pd.read_csv(url, index_col="Date", parse_dates=True)
X=X[X.Volume>0]
X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2316 entries, 2013-01-21 to 2022-07-27
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2316 non-null   float64
 1   High       2316 non-null   float64
 2   Low        2316 non-null   float64
 3   Close      2316 non-null   float64
 4   Adj Close  2316 non-null   float64
 5   Volume     2316 non-null   float64
dtypes: float64(6)
memory usage: 126.7 KB


In [6]:
TRAIN_TEST_CUTOFF = '2020-2-11'
TRAIN_VALID_RATIO = 0.75
    
# get the names of columns
cols = X.columns
    
# Save the target variable as a column in dataframe and drop NaN values
X["Target"] = (X["Close"].pct_change().shift(-1) > 0).astype(int)
X.dropna(inplace=True)
   
# Fit the standard scaler using the training dataset not whole data set
index = X.index[X.index > TRAIN_TEST_CUTOFF]
index = index[:int(len(index) * TRAIN_VALID_RATIO)]
scaler = StandardScaler().fit(X.loc[index, cols])
    
# Save scale transformed dataframe
X[cols] = scaler.transform(X[cols])
data = X

In [7]:
data['RoM3']=data.Close.rolling(3).mean() #Rolling Mean Closing Values of 3 days
data['RoM5']=data.Close.rolling(5).mean() #Rolling Mean Closing Values of 5 days
data['RoM15']=data.Close.rolling(15).mean() #Rolling Mean Closing Values of 15 days
data['RoM30']=data.Close.rolling(30).mean() #Rolling Mean Closing Values of 30 days
data['Vol_Diff'] = data["Volume"].pct_change().shift(-1) #Daily Volume Difference
data['Close_Diff'] = data["Close"].pct_change().shift(-7) #Weekly Closing Value Difference

#Dropping NaN rows
data.dropna(inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2280 entries, 2013-03-01 to 2022-07-18
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Open        2280 non-null   float64
 1   High        2280 non-null   float64
 2   Low         2280 non-null   float64
 3   Close       2280 non-null   float64
 4   Adj Close   2280 non-null   float64
 5   Volume      2280 non-null   float64
 6   Target      2280 non-null   int64  
 7   RoM3        2280 non-null   float64
 8   RoM5        2280 non-null   float64
 9   RoM15       2280 non-null   float64
 10  RoM30       2280 non-null   float64
 11  Vol_Diff    2280 non-null   float64
 12  Close_Diff  2280 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 249.4 KB


# Testing

Testing the 2dCNNpred models (64 in total) generated on Accuracy, Mean Absolute Error and Macro Averaged F1 Score. Hyperparameters that can be varied epochs, optimizer, batch size, droput rate, loss fuction.

In [8]:
param_list = [
 ['sgd', 'mae', 0.05], ['adam', 'mae', 0.1], ['adagrad', 'mae',  0.05], ['adamax', 'binary_focal_crossentropy', 0.1], 
 ['sgd', 'binary_focal_crossentropy', 0.1], ['adam', 'binary_focal_crossentropy', 0.15], ['adagrad', 'mae', 0.1], ['adamax', 'binary_focal_crossentropy', 0.15], 
 ['sgd', 'binary_crossentropy', 0.15], ['adam', 'binary_crossentropy', 0.2], ['adagrad', 'hinge', 0.2], ['adamax', 'binary_crossentropy', 0.15],
 ['sgd', 'hinge', 0.2], ['adam', 'hinge',  0.05], ['adagrad', 'hinge',  0.05], ['adamax', 'binary_crossentropy', 0.2]
 ]

id=1

df_eval=pd.DataFrame(columns=['id', 'optimizing_algo', 'loss_function','dropout_rate', 'batch_size', 'epochs', 'accuracy', 'MAE', 'F1'])


for batch_size in [64, 32]:
  for n_epochs in [20, 30]:
    for optim, lossfxn, dr in param_list:
   
      #load model for testing
      model = tf.keras.models.load_model('2dCNNpredNIFTY50m{}.h5'.format(id), compile=False)     
      model.compile(optimizer=optim, loss=lossfxn, metrics=["acc", f1macro])
      
      
      # Prepare test data
      test_data, test_target = testgen(data, 60, "Target")
      
      # Test the model
      test_out = model.predict(test_data)
      test_pred = (test_out > 0.5).astype(int)
      

      #measuring and saving performance
      df_eval.loc[len(df_eval)]=[id, optim, lossfxn, dr, batch_size, n_epochs, accuracy_score(test_pred, test_target), mean_absolute_error(test_pred, test_target), f1_score(test_pred, test_target)]

      output.clear()      
      print(id)

      #index of model
      id = id + 1
      

64


In [10]:
df_eval.to_csv('CNNpred_NIFTY50_performances.csv')
df_eval.head()

,id,optimizing_algo,loss_function,dropout_rate,batch_size,epochs,accuracy,MAE,F1
0,1,sgd,mae,0.05,64,20,0.543478,0.456522,0.702938
1,2,adam,mae,0.10,64,20,0.543478,0.456522,0.704225
2,3,adagrad,mae,0.05,64,20,0.541806,0.458194,0.702174
3,4,adamax,binary_focal_crossentropy,0.10,64,20,0.538462,0.461538,0.576687
4,5,sgd,binary_focal_crossentropy,0.10,64,20,0.530100,0.469900,0.551834


# Ensemble - Bagging

Ensemble-Bagging based on Hard Voting

In [12]:
test_pred=np.zeros((598,1))

# Prepare test data
test_data, test_target = testgen(data, 60, "Target")


for id in range (1,65):
  #load model for testing
  model = tf.keras.models.load_model('2dCNNpredNIFTY50m{}.h5'.format(id), compile=False)     
  model.compile(optimizer=optim, loss=lossfxn, metrics=["acc", f1macro])
      

  # Test the model
  test_out = model.predict(test_data)
  test_pred = np.add(test_pred, np.array((test_out > 0.5).astype(int)))
      

  #displaying progress
  output.clear() 
  print(id)

  #index of model
  id = id + 1

test_pred = (test_pred/id > 0.5).astype(int)

accuracy = accuracy_score(test_pred, test_target)
MAE = mean_absolute_error(test_pred, test_target)
F1 = f1_score(test_pred, test_target)

64


Ensemble accuracy 

In [13]:
print(accuracy)
print(MAE)
print(F1)

0.5451505016722408
0.45484949832775917
0.7049891540130151
